# inverted_indexes_cloab

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP. 

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met. 

In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.3 MB/s eta 0:00:00
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 124016 files and directories currently installed.)
Pre

In [3]:
import pyspark
from pyspark.sql import *
# from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [5]:
spark

## Copy some wiki data

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment. 

**IMPORTANT NOTE**

Before you start working with the wiki data, you first have to go over the 
"Working with GCP.pdf" Provided to you under the gcp folder in the same zip file as this folder. In that pdf you will redeem your credits for the GCP and create your instance. You have to do this procedure because the files will be mounted to you through GCP process. 
Notice that you have to change the project_id variable in the second cell below. 

In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it.


In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

# AUTH_TIMEOUT = 30

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  # signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except: 
  pass

In [7]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'homework3-370218'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


# Processing wikipedia

Now that we completed the setup and have some data in our local environment, we are ready to process it using PySpark. 

## A 2-minute intro to PySpark

One of the fundamental data structures in spark is a **Resillient Distributed Dataset (RDD)**. It is an immutable distributed collections of objects that are partitioned across machines, and thus making parallel processing natural and easy. RDD's support two types of operations: *transformations* (e.g. selecting a subset of fields from each element or projecting each element from A->B), and *actions* (e.g. save to disk or `collect` elements into the master node's memory). Watch out for `collect` operations that send too much data to the master node, it will crash without a trace because of running out of memory. 

Spark employs **lazy-evaluation** (like many of us...), meaning that it accumulates things we ask it to do into an execution plan, but doesn't do any calculation until a result is actually needed. When a result is needed? when we save something to disk, ask spark to show the data, or use other actions. But make no mistakes, Spark is not a slacker. It employs lazy-evaluation because it allows optimizations that wouldn't be possible otherwise and it often saves users from unnecessary data reads, tranfers across machines, and other costly operations. As we will see in a bit, the RDD data structure lends itself to simple and efficient MapReduce operations.

Let's look at our data before transforming it to RDD.

In [8]:
from pathlib import Path 
import os
import json
from collections import defaultdict

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

# לשנות את ההגבלה ל4000
parquetFile = spark.read.parquet(path)
doc_text_pairs = parquetFile.limit(4000).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(4000).select("title","id").rdd
doc_anchor_pairs = parquetFile.limit(4000).select("anchor_text").rdd

def map_func(doc_anchor):
  edge_set = []
  for id, anchor in doc_anchor[0]:
    if (id, anchor) not in edge_set:
      edge_set.append((id, anchor))
  return edge_set

doc_anchor_pairs_new = doc_anchor_pairs.flatMap(map_func).distinct().groupByKey().mapValues(list).map(lambda x:(" ".join(x[1]),x[0]))
# # parquetFile.show()

#more code
#convert doc_title to json - NEED TO RUN TO 8 MILLION FILES
# dictTitle=doc_title_pairs.collectAsMap()
# pv_clean = f'titles.json'
# with open(pv_clean, "w") as outfile:
#     json.dump(dictTitle, outfile)


# doc_title_pairs.repartition(1).write.csv(f'gs://{bucket_name}/titlePerDoc', compression="gzip")

# # Open the file for reading 
# with open('my_map.json', 'r') as f:
#  # Use json.load() to read the map from the 
#  file my_map = json.load(f)

#code for convert csv to json
# Open the file for reading 
# with open('output.json', 'r') as json_file: 
#   # Use json.load() to read the data from the file 
#   data = json.load(json_file) 
#   print(data)


# Select the second column of the RDD 
# names_column = doc_text_pairs.map(lambda x: x[1]).collect()

# ddd=defaultdict(list)
# for key in dic.keys():
#   for doc_id in dic[key]:
#     if(doc_id in names_column):
#       ddd[key].append(str(doc_id))
# print(ddd.items())



In [ ]:
# dict_items([('Most expensive city in the world', ['7780']),
#             ('India', ['2377']),
#             ('Apple computer', ['2275', '856', '2116', '1344', '2117']),
#             ('How to lose weight?', ['8460']),
#             ('how to deal with depression?', ['4531', '8389']),
#             ('Natural Language processing', ['5561']),
#             ('Dolly the sheep', ['6910']),
#             ('What is the best place to live in?', ['3138', '844', '8522', '3708', '5391', '5407', '1998'])])

In [62]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords =["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

#remember to change NUM_BUCKETS back to 124 
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  return([(k,(id,v)) for k,v in Counter(tokens).items() if k not in all_stopwords and v>0])

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key=lambda tup: tup[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda x:(x[0],len(x[1])))

  NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  dict1 = postings.map(lambda token: (token2bucket_id(token[0]), token)).groupByKey()
  return dict1.map(InvertedIndex.write_a_posting_list)

#merge the posting locations into a single dict and run more tests
def merge(postings):
  super_posting_locs = defaultdict(list)
  for posting_loc in postings:
    for k,v in posting_loc.items():
      super_posting_locs[k].extend(v)
  return super_posting_locs

def calc_dl(tokens,doc_id):
  return (doc_id,len(tokens))

def calc_total_term(postings):
  return postings.mapValues(helpFunc).collectAsMap()

def helpFunc(posting):
  count=0
  for tup in posting:
    count+=tup[1]
  return count

def doc_norm(text,doc_id):
  tokens=tokenize(text)
  dict_tokens=Counter(tokens)
  sum=0
  for term, tf in dict_tokens.items():
    sum+=tf*tf
  return (doc_id,math.sqrt(sum))


In [10]:
min_title=0
min_body=10
min_anchor=0
!mkdir body_index title_index anchor_index

In [64]:
# import zipfile 
#body
word_count_bodys = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
posting_lists_body = word_count_bodys.groupByKey().mapValues(reduce_word_counts)
filter_bodys = posting_lists_body.filter(lambda x: len(x[1])>min_body)
#global statistics
body_df=calculate_df(filter_bodys).collectAsMap()
posting_locs_body = partition_postings_and_write(filter_bodys).collect()
#merge the posting locations into a single dict and run more tests
super_posting_locs_body = defaultdict(list)
for posting_loc in posting_locs_body:
  for k,v in posting_loc.items():
    super_posting_locs_body[k].extend(v)


body_total_term=calc_total_term(filter_bodys)
body_dl=doc_text_pairs.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
body_norm=doc_text_pairs.map(lambda x: doc_norm(x[0],x[1])).collectAsMap()
print(body_norm)


#######index assignment 4 method##############
# doc_text_dict=doc_text_pairs.collectAsMap()
# # print(doc_text_pairs.collect()[1])
# body_data_text = {v: tokenize(k) for k,v in doc_text_dict.items()}
# index_text = InvertedIndex(docs=body_data_text)
# index_text.write('body_index','body')

# doc_title_dict=doc_title_pairs.collectAsMap()
# # print(doc_text_pairs.collect()[1])

# title_data_text = {v: tokenize(k) for k,v in doc_title_dict.items()}
# title_data_text = {v:k for k,v in doc_title_dict.items() if k != ''}
# # print(title_data_text)
# index_title = InvertedIndex(docs=title_data_text)
# index_title.write('title_index','title')


#title
# word_count_titles = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# posting_lists_title = word_count_titles.groupByKey().mapValues(reduce_word_counts)
# filter_titles = posting_lists_title.filter(lambda x: len(x[1])>min_title)
# #global statistics
# title_df=calculate_df(filter_titles).collectAsMap()
# posting_locs_title = partition_postings_and_write(filter_titles).collect()
# title_total_term=calc_total_term(filter_titles)
# title_dl=doc_title_pairs.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
# #merge the posting locations into a single dict and run more tests
# super_posting_locs_title = defaultdict(list)
# for posting_loc in posting_locs_title:
#   for k,v in posting_loc.items():
#     super_posting_locs_title[k].extend(v)

# #Anchor
# word_count_anchors = doc_anchor_pairs_new.flatMap(lambda x: word_count(x[0], x[1]))
# posting_lists_anchor = word_count_anchors.groupByKey().mapValues(reduce_word_counts)
# filter_anchors = posting_lists_anchor.filter(lambda x: len(x[1])>min_anchor)
# #global statistics
# anchor_df=calculate_df(filter_anchors).collectAsMap()
# posting_locs_anchor = partition_postings_and_write(filter_anchors).collect()
# anchor_total_term=calc_total_term(filter_anchors)
# anchor_dl=doc_anchor_pairs_new.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
#merge the posting locations into a single dict and run more tests
# super_posting_locs_anchor = defaultdict(list)
# for posting_loc in posting_locs_anchor:
#   for k,v in posting_loc.items():
#     super_posting_locs_anchor[k].extend(v)

#body
index_body=InvertedIndex()
index_body.posting_locs=super_posting_locs_body
index_body.df=body_df

index_body.DL=body_dl
index_body.norma=body_norm
index_body.term_total=body_total_term

#title
# index_title=InvertedIndex()
# index_title.posting_locs=super_posting_locs_title
# index_title.df=title_df
# index_title.DL=title_dl
# index_title.term_total=title_total_term

# #anchor
# index_anchor=InvertedIndex()
# index_anchor.posting_locs=super_posting_locs_anchor
# index_anchor.df=anchor_df
# index_anchor.DL=anchor_dl
# index_anchor.term_total=anchor_total_term

index_body.write_index('body_index', 'index_body')
# index_anchor.write_index('anchor_index', 'anchor')
# index_title.write('title_index', 'index_title')


# # Open the zip file in write mode 
# with zipfile.ZipFile('invertedAnchor.zip', 'w') as zip: 
# # Add the bin file to the zip file
#   for i in range(0,124):
#     string=str(i)+'_000.bin'
#     try: 
#       zip.write(string)
#     except:
#       continue


{12: 222.13284313671403, 25: 238.52043937574825, 39: 141.22322755127783, 290: 83.42661445845684, 303: 411.34535368714205, 305: 282.8851356999869, 307: 419.84878230143767, 308: 269.06318960422664, 309: 67.72739475278819, 316: 221.56714557894182, 324: 264.4806231087639, 330: 25.826343140289914, 332: 26.057628441590765, 334: 74.18894796396563, 336: 173.40703561274555, 339: 209.78560484456506, 340: 31.28897569432403, 344: 47.93745925682754, 358: 259.58428303732103, 359: 122.75585525749882, 569: 245.49541747250598, 572: 60.35726965329032, 573: 221.99324314041633, 579: 70.61161377563892, 580: 41.60528812542944, 586: 186.560981987124, 590: 45.89117562233506, 593: 234.21998206814038, 594: 631.2867811066536, 595: 368.38973927078916, 597: 164.07924914503965, 599: 190.7406616324899, 600: 260.8256122392891, 612: 78.65112840894274, 615: 74.60562981437795, 620: 198.022725968511, 621: 285.70614274110386, 624: 379.80652969637055, 627: 213.17129262637593, 628: 143.20963654726592, 630: 61.6765757804371,

In [ ]:
#SEARCH BACKEND

In [76]:
# %run createInvertedIndexUpdate
''''THIS FILE INCLUDES ALL THE METHODS USED BY SEARCH FUNCTIONS'''
import re
import nltk
from nltk.corpus import stopwords
from inverted_index_colab import *
import numpy as np
import math
from collections import defaultdict,Counter
from numpy.linalg import norm
# from search_frontend import *
import pandas as pd
import heapq


nltk.download('stopwords')


''''QUERY TOKANIZATION'''
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

#load the page rank json file
# with open ('pageRank.json','r') as f:
#   pageRank = json.load(f)

      
def queryRepresent(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    return list_of_tokens


''''BINARY SEARCH METHOD'''
def binarySearch(query,index):
  dict_results={}
  query=numpy.unique(query)
  for term in query:
    if term in index.df:
      postings=InvertedIndex.read_posting_list(index,term)
      for doc_id, tf in postings:
        dict_results[doc_id]=dict_results.get(doc_id,0)+1
  # sort result by the number of tokens appearance
  res = sorted(dict_results.items(), key=lambda x: x[1], reverse=True)
  # return [int(doc_id) for doc_id, score in res]
  res1=[]
  for tup in res:
    if str(tup[0]) in docs_titles.keys():
      res1.append((tup[0],docs_titles[str(tup[0])]))
    else:
      res1.append(tup[0])
  return res1
  # return [(tup[0], docs_titles[tup[0]]) for tup in res]


''''COSINE SIMILARITY METHOD'''
def generate_query_tfidf_vector(query_to_search, index):
    """
    Generate a vector representing the query. Each entry within this vector represents a tfidf score.
    The terms representing the query will be the unique terms in the index.

    We will use tfidf on the query as well.
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the query.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    Returns:
    -----------
    vectorized query with tfidf scores
    """

    epsilon = .0000001
    total_query_size = len(query_to_search)
    Q = numpy.zeros((total_query_size))
    counter = Counter(query_to_search)
    ind=0
    for token in numpy.unique(query_to_search):
        if token in index.term_total.keys():  # avoid terms that do not appear in the index.
            tf = counter[token] / len(query_to_search)  # term frequency divded by the length of the query
            df = index.df[token]
            idf = math.log((len(index.DL)) / (df + epsilon), 10)  # smoothing
            Q[ind] = tf * idf
            ind+=1
    return Q


def get_posting_iter(index):
    """
    This function returning the iterator working with posting list.

    Parameters:
    ----------
    index: inverted index
    """
    words, pls = zip(*index.posting_lists_iter())
    return words, pls

def get_candidate_documents_and_scores(query_to_search, index, path):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    path: path to posting list.

    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score.
    """
    candidates = {}
    for term in numpy.unique(query_to_search):
        posting_list=InvertedIndex.read_posting_list(index,term)
        normlized_tfidf=[]
        for tpl in posting_list:
          tf=tpl[1]/index.DL[tpl[0]]
          idf=math.log(len(index.DL)/index.df[term],10)
          tfidf=tf*idf
          normlized_tfidf.append((tpl[0],tfidf))
        # normlized_tfidf = [(tup[0],(tup[1]/index.DL[str(tup[0])])*math.log(len(index.DL)/index.df[term],10)) for tup in posting_list]
        for doc_id, tfidf in normlized_tfidf:
            candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf               

    return candidates
    #     normlized_tfidf=[]
    #     for doc_id, tf in posting_list:
    #       try:
    #         normlized_tfidf.append((doc_id, (tf / index.DL[str(doc_id)]) * math.log(len(index.DL) / index.df[term], 10))) 
    #         print(normlized_tfidf)
    #       except:
    #         normlized_tfidf.append((doc_id,0.0))
    #     for doc_id, tfidf in normlized_tfidf:
    #         candidates[(doc_id, term)] = candidates.get((doc_id, term), 0) + tfidf
    # return candidates


def cosine_similarity(Doc_vector, Q_vector,norma):
    """
    Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
    Generate a dictionary of cosine similarity scores
    key: doc_id
    value: cosine similarity score

    Parameters:
    -----------
    D: DataFrame of tfidf scores.

    Q: vectorized query with tfidf scores

    Returns:
    -----------
    dictionary of cosine similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: cosine similarty score.
    """
    # YOUR CODE HERE
    Q_array=np.array(Q_vector)
    doc_array=np.array(Doc_vector)
    denominator = norm(Q_vector)*norma
    if denominator==0.0:
      cosine=0.0
    else:
      cosine = numpy.dot(Q_vector, Doc_vector) / denominator
    return cosine

def top_n_cosine_sim(query,index,bodyPath,N=100):
    invertedIndex_term=get_candidate_documents_and_scores(query,index,bodyPath)
    #init all sim (q,d)=0
    dict_docs={}
    for doc, term in invertedIndex_term.keys():
      dict_docs[doc]=[0]*len(query)
    #init query vector tfidf
    query_vector=generate_query_tfidf_vector(query,index)
    #init term query index
    unique_terms={}
    i=0
    #dict of term idx in query
    for term in query:
      unique_terms[term]=i
      i+=1
    #init tfidf for doc_id,term
    for doc_id, term in invertedIndex_term.keys():
      indexTerm=unique_terms[term]
      tfidf=invertedIndex_term[(doc_id,term)]
      dict_docs[doc_id][indexTerm]=tfidf
    #create heap for tfidf scores
    top_n_docs=[]
    for doc, doc_tfidf_vector in dict_docs.items():
      cos_sim=cosine_similarity(doc_tfidf_vector,query_vector,index.norma[doc])
      if len(top_n_docs)==N:
        heapq.heapify(top_n_docs)
        mini=heapq.heappop(top_n_docs)
        if cos_sim>mini[0]:
          heapq.heappush(top_n_docs,(cos_sim,doc))
        else:
          heapq.heappush(top_n_docs,mini)
      else:
        heapq.heapify(top_n_docs)
        heapq.heappush(top_n_docs,(cos_sim,doc))
      top_n_docs=list(top_n_docs)
    #sorted scores results
    listi=sorted([(int(tup[1]),int(tup[0])) for tup in top_n_docs], key=lambda x: x[1], reverse=True)
    return [(tup[0],docs_titles[str(tup[0])]) for tup in listi]
    # return sorted([int(doc_id) for score, doc_i in top_n_docs], key=lambda x: x[1], reverse=True)
   
   #return sorted([(int(doc_id), docs_titles[str(doc_id)]) for score, doc_id in top_n_docs], key=lambda x: x[1], reverse=True)


''''BM25 METHOD'''
class BM25_from_index:
    """
    Best Match 25.
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self, index, path, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.path = path
        self.N = len(self.index.DL)
        self.AVGDL = sum(index.DL.values()) / self.N

    def calc_idf(self, list_of_tokens):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']

        Returns:
        -----------
        idf: dictionary of idf scores. As follows:
                                                    key: term
                                                    value: bm25 idf score
        """
        idf = {}
        for term in list_of_tokens:
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass
        return idf

    def get_candidate_documents_BM25(self,query,path,index):
        """
        Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
        and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
        Then it will populate the dictionary 'candidates.'
        For calculation of IDF, use log with base 10.
        tf will be normalized based on the length of the document.

        Parameters:
        -----------
        query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                         Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

        index:           inverted index loaded from the corresponding files.

        words,pls: iterator for working with posting.

        Returns:
        -----------
        dictionary of candidates. In the following format:
                                                                   key: pair (doc_id,term)
                                                                   value: tfidf score.
        """
        candidates = []
        for term in np.unique(query):
            if term in index.df:
                candidates+=InvertedIndex.read_posting_list(self.index,term)
            else:
                continue
        res=defaultdict(int)
        for doc, tf in candidates:
            res[doc]+=int(tf)
        tresh=10*len(query)
        return [doc_id for doc_id,tf in res.items() if tf>tresh]

    def search(self, query,N=100):
        """
        This function calculate the bm25 score for given query and document.
        We need to check only documents which are 'candidates' for a given query.
        This function return a dictionary of scores as the following:
                                                                    key: query_id
                                                                    value: a ranked list of pairs (doc_id, score) in the length of N.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        # YOUR CODE HERE
        scores = {}
        self.idf = self.calc_idf(query)
        term_tf={}
        for term in query:
            if term in self.index.df:
                # dict: key - term value - dict posting list of term (key - doc_id , value - tf)
                term_tf[term] = dict(InvertedIndex.read_posting_list(self.index,term))
        docs = self.get_candidate_documents_BM25(query,path,self.index)
        docList = []
        for doc in docs:
            docList.append((doc,self._score(query, doc,term_tf)))
         #sort docScores by scores
        scores = sorted(docList,key=lambda x: x[1],reverse=True)[:N]
        return [(doc[0],docs_titles[str(doc[0])]) for doc in scores]

    def _score(self, query, doc_id,term_tf):
        """
        This function calculate the bm25 score for given query and document.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        with open('pageRank.json', 'r') as f:
          page_rank = json.load(f)

        score = 0.0
        doc_len = self.index.DL[doc_id]
        for term in query:
            if doc_id in term_tf[term]:
                tfij = term_tf[term][doc_id]
                numerator = self.idf[term] * tfij * (self.k1 + 1)
                B = 1 - self.b + self.b * doc_len / self.AVGDL
                denominator = tfij + self.k1 * B
                if(float(page_rank[str(doc_id)])>0): #log cant get negatives or zeros
                    score += (numerator / denominator) + math.log(float(page_rank[str(doc_id)]))
                else:
                    score += (numerator / denominator)
        return score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#SEARCH FRONTEND

In [67]:
from flask import Flask, request, jsonify
from inverted_index_colab import *
import google
import numpy
from pandas import *
import json
     
#read the indexes from pkl files
titlePath=''
bodyPath='body_index'
anchorPath=''
body_index = InvertedIndex.read_index('body_index', 'index_body')
# title_index = InvertedIndex.read_index('title_index', 'title')
# anchor_index = InvertedIndex.read_index('anchor_index', 'anchor')


with open('titles.json','r') as f:
  docs_titles=json.load(f)

# class MyFlaskApp(Flask):
#     def run(self, host=None, port=None, debug=None, **options):
#         super(MyFlaskApp, self).run(host=host, port=port, debug=debug, **options)

# app = MyFlaskApp(__name__)
# app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False


# @app.route("/search")
def search(query):
    ''' Returns up to a 100 of your best search results for the query. This is 
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the 
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use 
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
#    query = request.args.get('query', '')
    # if len(query) == 0:
    #   return jsonify(res)
    # BEGIN SOLUTION
    query=queryRepresent(query)
    bm25=BM25_from_index(body_index,bodyPath)
    res=bm25.search(query)
    # END SOLUTION
    return res
    # return jsonify(res)

# @app.route("/search_body")
def search_body(query):
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
#    query = request.args.get('query', '')
    if len(query) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    query=queryRepresent(query)
    res=top_n_cosine_sim(query,body_index,'body_index',N=100)
    # END SOLUTION
#    return jsonify(res)
    return res

# @app.route("/search_title")
def search_title(query):
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE TITLE of articles, ordered in descending order of the NUMBER OF 
        DISTINCT QUERY WORDS that appear in the title. DO NOT use stemming. DO 
        USE the staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. For example, a document 
        with a title that matches two distinct query words will be ranked before a 
        document with a title that matches only one distinct query word, 
        regardless of the number of times the term appeared in the title (or 
        query). 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_title?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''
    res = []
    # query = request.args.get('query', '')
    # if len(query) == 0:
    #   return jsonify(res)
    # BEGIN SOLUTION
    query=queryRepresent(query)
    res=binarySearch(query,title_index)
    # END SOLUTION
    return res
    # return jsonify(res)

# @app.route("/search_anchor")
def search_anchor(query):
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE ANCHOR TEXT of articles, ordered in descending order of the 
        NUMBER OF QUERY WORDS that appear in anchor text linking to the page. 
        DO NOT use stemming. DO USE the staff-provided tokenizer from Assignment 
        3 (GCP part) to do the tokenization and remove stopwords. For example, 
        a document with a anchor text that matches two distinct query words will 
        be ranked before a document with anchor text that matches only one 
        distinct query word, regardless of the number of times the term appeared 
        in the anchor text (or query). 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_anchor?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''
    res = []
    # query = request.args.get('query', '')
    # if len(query) == 0:
    #   return jsonify(res)
    # BEGIN SOLUTION
    query=queryRepresent(query)
    res=binarySearch(query,anchor_index)
    # END SOLUTION
    return res
    # return jsonify(res)

# @app.route("/get_pagerank", methods=['POST'])
def get_pagerank(list):
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
    # wiki_ids = request.get_json()
    # if len(wiki_ids) == 0:
    #   return jsonify(res)
    # BEGIN SOLUTION
    with open('pageRank.json', 'r') as f:
      page_rank = json.load(f)
    for doc_id in list:
      if str(doc_id) in page_rank:
        res.append(page_rank[str(doc_id)])
    # END SOLUTION
    return res
    # return jsonify(res)

# @app.route("/get_pageview", methods=['POST'])
def get_pageview(list):
    ''' Returns the number of page views that each of the provide wiki articles
        had in August 2021.

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pageview
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pageview', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    '''
    res = []
    # wiki_ids = request.get_json()
    # if len(wiki_ids) == 0:
    #   return jsonify(res)
    # BEGIN SOLUTION
    page_view = pandas.read_pickle(r'pageviews-202108-user.pkl')
    for doc_id in list:
      if doc_id in page_view:
        res.append(page_view[str(doc_id)])
    # END SOLUTION
    return res


# if __name__ == '__main__':
#     # run the Flask RESTful API, make the server publicly available (host='0.0.0.0') on port 8080
#     app.run(host='0.0.0.0', port=8080, debug=True)


In [78]:
#query="What is the best place to live in?"
query='Apple computer'
#query='how to deal with depression?'
# print(search(query))
# print(search_body(query))
# print('---body---')

# print(doc_text_pairs.collect()[0])

# print('---title---')

# print(doc_title_pairs.collect()[0])

# print('---anchor before change---')

# print(doc_anchor_pairs.collect()[0])

# print('---anchor after change---')

# print('---location 0---')

# print(doc_anchor_pairs_new.collect()[0])
predicted_list=search_body(query)
print(predicted_list)
#true_list=['3138', '844', '8522', '3708', '5391', '5407', '1998']
true_list=['2275', '856', '2116', '1344', '2117']
#true_list=['4531', '8389']
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    print(true_set)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):
        print(doc_id[0])    
        if str(doc_id[0]) in true_set:
            print(1)
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)
print(average_precision(true_list, predicted_list))
#print(search_anchor(query))
# print('---location 1---')
# print(get_pageview([2377,3138,856]))
# print(doc_anchor_pairs_new.collect()[1])
#print(get_pagerank([1,2,2377,395763]))
#print(get_pageview([1,2,2377,395763]))
# dict_items([('Most expensive city in the world', ['7780']),
#             ('India', ['2377']),
#             ('Apple computer', ['2275', '856', '2116', '1344', '2117']),
#             ('How to lose weight?', ['8460']),
#             ('how to deal with depression?', ['4531', '8389']),
#             ('Natural Language processing', ['5561']),
#             ('Dolly the sheep', ['6910']),
#             ('What is the best place to live in?', ['3138', '844', '8522', '3708', '5391', '5407', '1998'])])

[(2959, 'Arcadia 2001'), (2332, 'AD (disambiguation)'), (1169, 'APL'), (4825, 'BeBox'), (6603, 'Case'), (2935, 'ANS'), (3823, 'Binary'), (2115, 'AppleTalk'), (2137, 'Aster CT-80'), (8527, 'Discordianism'), (1404, 'AV'), (6559, 'Control store'), (2710, 'Au'), (7962, 'Logical disjunction'), (630, 'Ada'), (887, 'MessagePad'), (2316, 'Audio file format'), (6974, 'Computer music'), (4772, 'BBS'), (7952, 'Digital Equipment Corporation'), (2900, 'File archiver'), (5323, 'Computer science'), (6456, 'Charles Edward Jones'), (3114, 'Amiga 500 Plus'), (856, 'Apple Inc.'), (4352, 'Backplane'), (4051, 'Brian Kernighan'), (8218, 'Dennis Ritchie'), (1194, 'Atomic'), (2950, 'Anyone Can Whistle'), (4801, 'BeOS'), (6626, 'CDE'), (7535, 'Commodore'), (2170, 'ABCD'), (2116, 'Apple II series'), (8000, 'Default'), (4498, 'Benchmark'), (8386, 'Dynamics'), (2142, 'List of artificial intelligence projects'), (7253, 'Cartesian'), (6080, 'CGI'), (6094, 'CPD'), (1349, 'Atanasoff–Berry computer'), (7331, 'Cellular